For TPU

In [ ]:
!pip install tensorflow==2.18.0
!pip install tensorflow-tpu==2.18.0 --find-links=https://storage.googleapis.com/libtpu-tf-releases/index.html

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 MB 586.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml_dtypes 0.5.1
    Uninstalling ml_dtypes-0.5.1:
      Successfully uninstalled ml_dtypes-0.5.1
Looking in links: https://storage.googleapis.com/libtpu-tf-releases/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/1

Importing libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
from google.colab import drive
from tensorflow.keras import layers, models, callbacks
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.11/dist-packages/jax/__init__.py:31: UserWarning: cloud_tpu_init failed: AttributeError("module 'libtpu' has no attribute 'get_library_path'")
 This a JAX bug; please report an issue at https://github.com/jax-ml/jax/issues
  _warn(f"cloud_tpu_init failed: {exc!r}\n This a JAX bug; please report "


Importing data

In [18]:
drive.mount('/content/drive')
'''
# Paths to LR and HR folders
lr_path = '/content/drive/My Drive/Lensing Resources/Dataset/LR'
hr_path = '/content/drive/My Drive/Lensing Resources/Dataset/HR'


# Function to load images from a folder and convert them to NumPy arrays
def load_images_to_numpy(folder_path):
    images = []
    for filename in os.listdir(folder_path):
          img_path = os.path.join(folder_path, filename)
          img_array = np.load(img_path)
          images.append(img_array)
    return np.array(images)

lr_images = load_images_to_numpy(lr_path)
hr_images = load_images_to_numpy(hr_path)
'''
#This takes too much time, trying to load zip and extract here. Should be faster.

import zipfile

zip_path = '/content/drive/My Drive/Lensing Resources/3b_Dataset.zip'

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/3b_Dataset')

# Load all .npy files from the extracted folder
lr_test_path = '/content/3b_Dataset/B LR Test.npy'
lr_val_path = '/content/3b_Dataset/B LR val.npy'
lr_train_path = '/content/3b_Dataset/B LR train.npy'
hr_test_path = '/content/3b_Dataset/B HR test.npy'
hr_val_path = '/content/3b_Dataset/B HR val.npy'
hr_train_path = '/content/3b_Dataset/B HR train.npy'

lr_test_images = np.load(lr_test_path)
hr_test_images = np.load(hr_test_path)
lr_train_images = np.load(lr_train_path)
hr_train_images = np.load(hr_train_path)
lr_val_images = np.load(lr_val_path)
hr_val_images = np.load(hr_val_path)

# Convert from channels-first to channels-last format
lr_test_images = np.transpose(lr_test_images, (0, 2, 3, 1))
hr_test_images = np.transpose(hr_test_images, (0, 2, 3, 1))
lr_train_images = np.transpose(lr_train_images, (0, 2, 3, 1))
hr_train_images = np.transpose(hr_train_images, (0, 2, 3, 1))
lr_val_images = np.transpose(lr_val_images, (0, 2, 3, 1))
hr_val_images = np.transpose(hr_val_images, (0, 2, 3, 1))

# resize LR from 64×64→75×75
lr_test_images  = tf.image.resize(lr_test_images,  [75, 75], method='bicubic').numpy()
lr_train_images = tf.image.resize(lr_train_images, [75, 75], method='bicubic').numpy()
lr_val_images   = tf.image.resize(lr_val_images,   [75, 75], method='bicubic').numpy()

# resize HR from 128×128→150×150
hr_test_images  = tf.image.resize(hr_test_images,  [150, 150], method='bicubic').numpy()
hr_train_images = tf.image.resize(hr_train_images, [150, 150], method='bicubic').numpy()
hr_val_images   = tf.image.resize(hr_val_images,   [150, 150], method='bicubic').numpy()


# Split into training and validation sets
X_train, X_val, y_train, y_val = lr_train_images, lr_val_images, hr_train_images, hr_val_images

# Just verifying stuff
print("LR images shape:", lr_test_images.shape)
print("HR images shape:", hr_test_images.shape)
print(f"Number of LR images: {len(lr_test_images)}")
print(f"Number of HR images: {len(hr_test_images)}")
print(f"Shape of one LR image: {lr_test_images[0].shape}")
print(f"Shape of one HR image: {hr_test_images[0].shape}")
print("Sample entry", lr_test_images[0])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
LR images shape: (27, 75, 75, 1)
HR images shape: (27, 150, 150, 1)
Number of LR images: 27
Number of HR images: 27
Shape of one LR image: (75, 75, 1)
Shape of one HR image: (150, 150, 1)
Sample entry [[[0.08862138]
  [0.05581525]
  [0.03803296]
  ...
  [0.047352  ]
  [0.0398311 ]
  [0.03235132]]

 [[0.06973191]
  [0.05420327]
  [0.05125581]
  ...
  [0.03537383]
  [0.03695184]
  [0.05092319]]

 [[0.05120282]
  [0.06054492]
  [0.07637955]
  ...
  [0.02066625]
  [0.02027351]
  [0.04833489]]

 ...

 [[0.03380245]
  [0.04052979]
  [0.04019793]
  ...
  [0.02767885]
  [0.03222917]
  [0.04045255]]

 [[0.03955572]
  [0.04187552]
  [0.03708259]
  ...
  [0.03438776]
  [0.04369233]
  [0.04964432]]

 [[0.03955173]
  [0.0341731 ]
  [0.03482848]
  ...
  [0.04559111]
  [0.04222794]
  [0.06340541]]]


Defining RCAN

In [21]:
import sys

# Increase recursion limit for deep architectures
sys.setrecursionlimit(10000)

class DepthToSpaceLayer(layers.Layer):
    def __init__(self, block_size, **kwargs):
        super().__init__(**kwargs)
        self.block_size = block_size

    def call(self, inputs):
        return tf.nn.depth_to_space(inputs, self.block_size)

    def get_config(self):
        return {'block_size': self.block_size}

def rcab(inputs, num_filters, reduction=16):
    # Residual Channel Attention Block
    x = layers.Conv2D(num_filters, 3, padding='same', activation='relu')(inputs)
    x = layers.Conv2D(num_filters, 3, padding='same')(x)

    # Channel attention
    ca = layers.GlobalAveragePooling2D(keepdims=True)(x)
    ca = layers.Conv2D(num_filters//reduction, 1, activation='relu')(ca)
    ca = layers.Conv2D(num_filters, 1, activation='sigmoid')(ca)

    x = layers.Multiply()([x, ca])
    return layers.Add()([x, inputs])

def residual_group(x, num_rcab, num_filters):
    # Residual Group with multiple RCABs
    for _ in range(num_rcab):
        x = rcab(x, num_filters)
    return layers.Conv2D(num_filters, 3, padding='same')(x)

def build_rcan(input_shape=(75, 75, 1), scale=2, num_rg=10, num_rcab=20, num_filters=64):
    inputs = layers.Input(shape=input_shape)

    # Shallow feature extraction
    x_init = layers.Conv2D(num_filters, 3, padding='same')(inputs)

    # Deep feature extraction
    x = x_init
    for _ in range(num_rg):
        # Residual Group with skip connection
        rg = residual_group(x, num_rcab, num_filters)
        x = layers.Add()([x, rg])

    # Global skip connection
    x = layers.Conv2D(num_filters, 3, padding='same')(x)
    x = layers.Add()([x, x_init])

    # Upsampling
    x = layers.Conv2D(num_filters * (scale ** 2), 3, padding='same')(x)
    x = DepthToSpaceLayer(scale)(x)

    # Reconstruction
    outputs = layers.Conv2D(1, 3, padding='same', activation='relu')(x)

    return models.Model(inputs, outputs, name='RCAN')

Compile

In [22]:
import tensorflow as tf
from tensorflow.keras.saving import register_keras_serializable
from tensorflow.keras.models import load_model

# Register the custom layer
@register_keras_serializable()
class DepthToSpace(tf.keras.layers.Layer):
    def __init__(self, scale, **kwargs):
        super(DepthToSpace, self).__init__(**kwargs)
        self.scale = scale

    def call(self, inputs):
        return tf.nn.depth_to_space(inputs, self.scale)

    def get_config(self):
        config = super(DepthToSpace, self).get_config()
        config.update({'scale': self.scale})
        return config

# Now load the model with the custom layer
model_path = '/content/drive/MyDrive/Lensing Resources/3a_best_checkpoint.keras'

model = load_model(model_path, custom_objects={'DepthToSpace': DepthToSpace})

# Continue with checkpointing, training etc.
from tensorflow.keras import callbacks

checkpoint_path = '/content/drive/MyDrive/Lensing Resources/3b_checkpointing.keras'

checkpoint_cb = callbacks.ModelCheckpoint(
    checkpoint_path,
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,
    mode='min'
)


Train

In [23]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[checkpoint_cb]
)

Epoch 1/50


NotFoundError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 510, in dispatch_queue

  File "/usr/local/lib/python3.11/dist-packages/ipy

Conclusion

In [ ]:
best_model = models.load_model(checkpoint_path)
loss, mae = best_model.evaluate(X_val, y_val)
print(f'Validation Loss: {loss:.4f}, MAE: {mae:.4f}')

def plot_results(lr, hr, sr, index):
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 3, 1)
    plt.title('Low Resolution')
    plt.imshow(lr[index].squeeze(), cmap='gray')

    plt.subplot(1, 3, 2)
    plt.title('High Resolution')
    plt.imshow(hr[index].squeeze(), cmap='gray')

    plt.subplot(1, 3, 3)
    plt.title('Super Resolution')
    plt.imshow(sr[index].squeeze(), cmap='gray')
    plt.show()

# Generate super-resolution images
sr_images = best_model.predict(X_val)

# Plot some results
plot_results(X_val, y_val, sr_images, index=0)
plot_results(X_val, y_val, sr_images, index=1)